In [183]:
import pandas as pd

In [184]:
files = ['data/amazon_cells_labelled.txt','data/imdb_labelled.txt','data/yelp_labelled.txt']

In [185]:
df = pd.DataFrame({'data':[],'label':[]})
for f in files:
    frame = pd.read_csv(f,sep='\t',names=['data','label'])
    df = pd.concat([df,frame],axis=0)

In [186]:
df

,data,label
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,Tied to charger for conversations lasting more...,0.0
4,The mic is great.,1.0
...,...,...
995,I think food should have flavor and texture an...,0.0
996,Appetite instantly gone.,0.0
997,Overall I was not impressed and would not go b...,0.0
998,"The whole experience was underwhelming, and I ...",0.0


In [187]:
from keras.preprocessing.text import Tokenizer

In [188]:
t = Tokenizer()

In [189]:
t.fit_on_texts(df['data'])

In [190]:
X = t.texts_to_sequences(df['data'])
Y = df['label']

In [191]:
vocab_size = len(t.word_index)+1

In [192]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [201]:
max_len = 50
X = pad_sequences(X,padding='post',maxlen = max_len)

In [202]:
Y=df['label']

In [203]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten, BatchNormalization

In [204]:
import numpy as np
def create_embedding_matrix(file,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(file,encoding='utf-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx]=np.array(vector)
    return embedding_matrix

In [205]:
embedding_matrix = create_embedding_matrix('data/glove.6B.50d.txt',t.word_index,50)

In [206]:
from tensorflow.keras.backend import clear_session
clear_session()
model = Sequential(
[
    Embedding(input_dim =vocab_size,weights=[embedding_matrix],output_dim=50,input_length=max_len),
    LSTM(10,activation='tanh'),
    Dense(64, activation='tanh'),
    Dense(32, activation='tanh'),
    BatchNormalization(),    
    Dense(1, activation='sigmoid')
]
)

In [207]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001), metrics='accuracy')

In [208]:
model.fit(X,Y,validation_split=0.3, epochs=20,steps_per_epoch=10)

Epoch 1/20
10/10 [==============================] - 4s 120ms/step - loss: 0.6933 - accuracy: 0.4982 - val_loss: 0.6933 - val_accuracy: 0.4861
Epoch 2/20
10/10 [==============================] - 1s 54ms/step - loss: 0.6751 - accuracy: 0.5840 - val_loss: 0.6890 - val_accuracy: 0.6679
Epoch 3/20
10/10 [==============================] - 1s 53ms/step - loss: 0.5796 - accuracy: 0.7051 - val_loss: 0.7096 - val_accuracy: 0.5139
Epoch 4/20
10/10 [==============================] - 1s 53ms/step - loss: 0.5185 - accuracy: 0.7748 - val_loss: 0.6814 - val_accuracy: 0.5200
Epoch 5/20
10/10 [==============================] - 1s 53ms/step - loss: 0.4652 - accuracy: 0.8050 - val_loss: 0.6838 - val_accuracy: 0.5200
Epoch 6/20
10/10 [==============================] - 1s 53ms/step - loss: 0.4183 - accuracy: 0.8279 - val_loss: 0.6682 - val_accuracy: 0.5588
Epoch 7/20
10/10 [==============================] - 1s 53ms/step - loss: 0.3701 - accuracy: 0.8534 - val_loss: 0.6496 - val_accuracy: 0.7430
Epoch 8/20
1